# Energy Out-of-distribution detection

From the paper [Energy-based Out-of-distribution Detection](http://arxiv.org/abs/2010.03759). Let's explore the folder structure, the models and the available scripts and extract what we need for our ideas.

In [13]:
#import sys
#sys.path.append("./energy_odd/CIFAR")
from energy_ood.CIFAR.models.wrn import WideResNet
import torch

In [22]:
model = WideResNet(40, 10, 2, dropRate=0.4)
model.load_state_dict(torch.load("energy_ood/CIFAR/snapshots/pretrained/cifar10_wrn_pretrained_epoch_99.pt", map_location=torch.device('cpu')))
model.eval()

WideResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (block1): NetworkBlock(
    (layer): Sequential(
      (0): BasicBlock(
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (convShortcut): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (1): BasicBlock(
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(32, 